In [15]:
import optuna

import lightgbm as lgb

from sklearn.datasets import make_regression

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error

In [16]:
# Generate sample regression data

X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)

In [17]:
# Split the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Define Optuna objective function

def objective(trial):

    params = {

    'learning_rate' :trial.suggest_float('learning_rate',0.01, 0.2),
    'num_leaves' : trial.suggest_int( 'num_leaves', 20, 50),
    'max_depth':trial.suggest_int('max_depth', 3,7),
    'n_estimators':trial.suggest_int('n_estimators',50, 200),
    "force_col_wise": True,  # Enforcing column-wise multi-threading
    'verbose': -1

    
    }

    model = lgb.LGBMRegressor(**params)

    model.fit(X_train,y_train)

    preds = model.predict(X_test)

    return mean_squared_error(y_test, preds)


In [19]:
# Run Optuna optimization

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=20)

[I 2025-02-14 15:27:45,045] A new study created in memory with name: no-name-9eb51c7b-a609-4d11-afd3-d04518564691
[I 2025-02-14 15:27:45,256] Trial 0 finished with value: 795.8015826516618 and parameters: {'learning_rate': 0.16987211728639523, 'num_leaves': 38, 'max_depth': 4, 'n_estimators': 133}. Best is trial 0 with value: 795.8015826516618.
[I 2025-02-14 15:27:45,486] Trial 1 finished with value: 978.4700564936743 and parameters: {'learning_rate': 0.1630323263428925, 'num_leaves': 26, 'max_depth': 5, 'n_estimators': 121}. Best is trial 0 with value: 795.8015826516618.
[I 2025-02-14 15:27:45,797] Trial 2 finished with value: 907.1245027348142 and parameters: {'learning_rate': 0.1754624144656909, 'num_leaves': 43, 'max_depth': 5, 'n_estimators': 165}. Best is trial 0 with value: 795.8015826516618.
[I 2025-02-14 15:27:46,069] Trial 3 finished with value: 875.8505060737612 and parameters: {'learning_rate': 0.1047660016765698, 'num_leaves': 27, 'max_depth': 5, 'n_estimators': 138}. Best

In [20]:
# Get best parameters

print(f'Best Parameters : {study.best_params}')

Best Parameters : {'learning_rate': 0.17067677967957348, 'num_leaves': 32, 'max_depth': 3, 'n_estimators': 179}


In [21]:
# Get best parameters

best_params = study.best_params

final_model = lgb.LGBMRegressor(**best_params)

final_model.fit(X_train,y_train)

y_pred = final_model.predict(X_test)

print(f'Final Model MSE : {mean_squared_error(y_test, y_pred)}')


Final Model MSE : 661.7610566056676
